# My own version of RMSD for RDKit compatibility. You can simply ignore it.

In [1]:
import numpy as np
import os, sys
if os.path.isfile(os.getenv("HOME") + '/software/ArbAlign/hungarian.cpython-36m-x86_64-linux-gnu.so'):
    sys.path.append(os.getenv("HOME") + '/software/ArbAlign/')
    import hungarian
    linear_sum_assignment =  hungarian.lap
else:
    from scipy.optimize import linear_sum_assignment

from scipy.spatial.distance import cdist

AXIS_SWAPS       = np.array([[0,1,2],[0,2,1],[1,0,2],[1,2,0],[2,0,1],[2,1,0]])
mask_swaps       =           [  1   ,  -1   ,  -1   ,   1   ,   1   ,  -1  ]
AXIS_REFLECTIONS = np.array([[1,1,1],[-1,1,1],[1,-1,1],[1,1,-1],[-1,-1,1],[-1,1,-1],[1,-1,-1],[-1,-1,-1]])
mask_reflections =           [  1   ,   -1   ,   -1   ,  -1    ,     1   ,    1    ,    1    ,    -1   ]
_rmsd = lambda V, W: np.sqrt(sum([(v[i]-w[i])**2.0 for i in range(len(V[0])) for v, w in zip(V, W)])/len(V))
centroid = lambda X: X.mean(axis=0)

# Extract element symbols and atomic coordinates from PDB string or filename
def parse_pdb(pdb, noHydrogens=True):
    """
    Reads an pdf file into lists
    filename - name of xyz file
    noHydrogens - if true, hydrogens are ignored; if not, hydrogens are included

    Returns a tuple (a, b)
    where a is a list of coordinate labels and b is a set of coordinates
    (i.e) a = ["O", "H", "H"], b = [[x0,y0,z0],[x1,y1,z1],[x2,y2,z2]]
    """
    pdb = pdb.split('\n')
    if len(pdb) == 1:
        with open(pdb[0], 'r') as fr:
            pdb = fr.readlines()
    labels = []
    coords = []
    for i in [i for i in pdb if i.find('HETATM') == 0 or i.find('ATOM') == 0 ]:
        # ATOM      1  OXT GLY A   3       2.261  -2.612   1.127  1.00  0.00           O
        # HETATM    1  C1  UNL     1      -7.227   0.602  -0.445  1.00  0.00           C
        ElementSymbol = i[76:78].strip().upper()
        #print(noHydrogens)
        if noHydrogens and ElementSymbol == "H":
            continue
        else:
            labels.append(ElementSymbol)
            coords.append([i[30:38],i[38:46],i[46:54]])
    #print(unsorted_labels, np.array(unsorted_coords, dtype=float), NA )
    return np.asarray(labels), np.array(coords, dtype=float)

# Rotate the coordinates of probe molecule according to those of reference molecule
def kabsch_rmsd(prb, ref):
    # Computation of the covariance matrix
    C = np.dot(np.transpose(prb), ref)

    # Computation of the optimal rotation matrix
    V, S, W = np.linalg.svd(C)

    # Ensure a right-handed coordinate system
    if (np.linalg.det(V) * np.linalg.det(W)) < 0.0:
        S[-1] = -S[-1]
        V[:, -1] = -V[:, -1]

    # Create Rotation matrix U
    U = np.dot(V, W)
    prb = np.dot(prb, U)
    return _rmsd(prb, ref), prb

def reorder_hungarian(p_atoms, q_atoms, p_coord, q_coord):
    """
    Re-orders the input atom list and xyz coordinates using the Hungarian
    method (using optimized column results)

    Parameters
    ----------
    p_atoms : array
        (N,1) matrix, where N is points holding the atoms' names
    p_atoms : array
        (N,1) matrix, where N is points holding the atoms' names
    p_coord : array
        (N,D) matrix, where N is points and D is dimension
    q_coord : array
        (N,D) matrix, where N is points and D is dimension

    Returns
    -------
    view_reorder : array
             (N,1) matrix, reordered indexes of atom alignment based on the
             coordinates of the atoms

    """

    # Find unique atoms
    unique_atoms = np.unique(p_atoms)

    # generate full view from q shape to fill in atom view on the fly
    view_reorder = np.zeros(q_atoms.shape, dtype=int)
    view_reorder -= 1
    for atom in unique_atoms:
        p_atom_idx, = np.where(p_atoms == atom)
        q_atom_idx, = np.where(q_atoms == atom)

        A_coord = p_coord[p_atom_idx]
        B_coord = q_coord[q_atom_idx]

        # should be kabasch here i think
        distances = cdist(A_coord, B_coord, 'euclidean')

        # Perform Hungarian analysis on distance matrix between atoms of 1st
        # structure and trial structure
        _, view = linear_sum_assignment(distances)

        view_reorder[p_atom_idx] = q_atom_idx[view]

    return view_reorder

# Description of the rotated molecule in XYZ format
def get_xyz(atoms, V, title="", decimals=8):
    """
    Print coordinates V with corresponding atoms to stdout in XYZ format.
    Parameters
    ----------
    atoms : list
        List of atomic types
    V : array
        (N,3) matrix of atomic coordinates
    title : string (optional)
        Title of molecule
    decimals : int (optional)
        number of decimals for the coordinates

    Return
    ------
    output : str
        Molecule in XYZ format

    """
    N, D = V.shape

    fmt = "{:2s}" + (" {:15."+str(decimals)+"f}")*3

    out = list()
    out += [str(N)]
    out += [title]

    for i in range(N):
        atom = atoms[i]
        atom = atom[0].upper() + atom[1:]
        out += [fmt.format(atom, V[i, 0], V[i, 1], V[i, 2])]

    return "\n".join(out)

def _align(ref_symbol, prb_symbol, ref_coord, prb_coord, ref_cent,
          reorder=False, keep_stereo=True):

    min_rmsd   = np.inf
    min_review = None
    tmp_review = None

    for swap, i in zip(AXIS_SWAPS, mask_swaps):
        for reflection, j in zip(AXIS_REFLECTIONS, mask_reflections):
            if keep_stereo and  i * j == -1: continue # skip enantiomers

            # Swap, reflect, and translate the probe molecule
            tmp_coord  = prb_coord[:, swap]
            tmp_coord  = np.dot(tmp_coord, np.diag(reflection))
            tmp_coord -= centroid(tmp_coord)

            # Reorder the modifed probe
            if reorder:
                tmp_review = reorder_hungarian(ref_symbol, prb_symbol, ref_coord, tmp_coord)
                tmp_coord = tmp_coord[tmp_review]

            # Rotation
            this_rmsd, tmp_coord  = kabsch_rmsd(tmp_coord, ref_coord)

            if this_rmsd < min_rmsd:
                min_rmsd    = this_rmsd
                min_review  = tmp_review
                min_coord   = tmp_coord
                i_j_product = i * j 

    if reorder:
        prb_symbol = prb_symbol[min_review]
    prb_coord = min_coord + ref_cent

    prb_xyz = get_xyz(prb_symbol, prb_coord, title="aligned", decimals=3)
    return min_rmsd, prb_xyz, i_j_product

def align(ref_pdb, prb_pdb, noHydrogens=True, reorder=False, keep_stereo=True):
    ref_symbol, ref_coord = parse_pdb(ref_pdb, noHydrogens)
    prb_symbol, prb_coord = parse_pdb(prb_pdb, noHydrogens)
    if len(ref_symbol) != len(prb_symbol): raise
    ref_cent = centroid(ref_coord)
    prb_cent = centroid(prb_coord)
    ref_coord -= ref_cent
    prb_coord -= prb_cent
    result = _align(ref_symbol, prb_symbol,
                    ref_coord, prb_coord, ref_cent,
                    reorder, keep_stereo)
    return result

if __name__ == "__main__":
    ref = '''REMARK   1 File created by GaussView 5.0.9
HETATM    1  C           0      -0.084  -0.121  -2.952                       C
HETATM    2  H           0       0.329  -1.099  -3.088                       H
HETATM    3  H           0       0.335   0.547  -3.675                       H
HETATM    4  H           0      -1.146  -0.162  -3.076                       H
HETATM    5  C           0       0.248   0.383  -1.535                       C
HETATM    6  H           0      -0.163   1.362  -1.400                       H
HETATM    7  H           0       1.310   0.422  -1.410                       H
HETATM    8  C           0      -0.357  -0.576  -0.494                       C
HETATM    9  H           0       0.053  -1.555  -0.629                       H
HETATM   10  H           0      -1.419  -0.615  -0.619                       H
HETATM   11  C           0      -0.025  -0.072   0.923                       C
HETATM   12  N           0      -0.589   1.273   1.109                       N
HETATM   13  H           0      -1.548   1.197   1.383                       H
HETATM   14  H           0      -0.077   1.754   1.821                       H
HETATM   15  O           0       1.394  -0.021   1.090                       O
HETATM   16  H           0       1.608  -0.064   2.025                       H
HETATM   17  F           0      -0.556  -0.913   1.836                       F
END '''
    prb = '''TITLE       Required
REMARK   1 File created by GaussView 5.0.9
HETATM    1  C           0     -16.365  -0.185   0.225                       C
HETATM    2  H           0     -17.102  -0.670  -0.381                       H
HETATM   11  C           0     -14.053  -3.247   0.772                       C
HETATM   12  N           0     -13.039  -2.580   1.602                       N
HETATM   13  H           0     -13.172  -2.836   2.560                       H
HETATM    9  H           0     -16.088  -2.904   0.178                       H
HETATM   10  H           0     -15.712  -2.333   1.785                       H
HETATM   16  H           0     -13.006  -4.136  -0.641                       H
#HETATM   17  F           0     -14.311  -4.478   1.268                       F
HETATM   17  F           0     -14.301  -4.478   1.268                       F
HETATM   14  H           0     -12.127  -2.862   1.306                       H
HETATM   15  O           0     -13.570  -3.362  -0.569                       O
HETATM    3  H           0     -16.167   0.791  -0.166                       H
HETATM    4  H           0     -16.728  -0.102   1.229                       H
HETATM    5  C           0     -15.067  -1.015   0.216                       C
HETATM    6  H           0     -14.330  -0.528   0.821                       H
HETATM    7  H           0     -14.705  -1.100  -0.787                       H
HETATM    8  C           0     -15.351  -2.418   0.781                       C
END '''
    min_rmsd, prb_xyz, i_j_product = align(ref, prb, noHydrogens=True, reorder=True, keep_stereo=False)
    print('mininum rmsd:',min_rmsd,'\n')
    print(prb_xyz)

mininum rmsd: 0.000373383768795183 

7
aligned
C           -0.084          -0.121          -2.952
C            0.248           0.383          -1.535
C           -0.357          -0.576          -0.494
C           -0.025          -0.072           0.923
N           -0.589           1.273           1.109
O            1.394          -0.021           1.090
F           -0.556          -0.913           1.836


# aRMSD/PdbOp1+2/M1.pdb as ref (reference molecule)
# aRMSD/PdbOp1+2/M2.pdb as prb (probe molecule)

In [2]:
ref = '''CRYST1    8.217   15.231   10.729  90.00  98.82  90.00 P 1 21 1      2
TITLE OLEX2 export (an aspirinate model derived from FEHGAB)
ATOM      1   O1                -1.750   9.273   1.274  1.00  3.95           O  
ATOM      2   O2                -0.995  10.748   2.718  1.00  3.95           O  
ATOM      3   C3                -1.805  10.421   1.772  1.00  3.95           C  
ATOM      4   C4                -2.791  11.443   1.319  1.00  3.95           C  
ATOM      5   C5                -3.163  11.574  -0.014  1.00  3.95           C  
ATOM      6   C6                -4.057  12.572  -0.398  1.00  3.95           C  
ATOM      7   H7                -4.316  12.657  -1.308  1.00  3.95           H  
ATOM      8   C8                -4.561  13.434   0.550  1.00  3.95           C  
ATOM      9   H9                -5.133  14.145   0.284  1.00  3.95           H  
ATOM     10  C10                -4.247  13.279   1.873  1.00  3.95           C  
ATOM     11  H11                -4.653  13.828   2.531  1.00  3.95           H  
ATOM     12  C12                -3.323  12.302   2.247  1.00  3.95           C  
ATOM     13  H13                -3.059  12.231   3.156  1.00  3.95           H  
ATOM     14  C14                -2.969   9.759  -1.491  1.00  3.95           C  
ATOM     15  C15                -2.137   9.107  -2.490  1.00  3.95           C  
ATOM     16  H16                -1.318   9.628  -2.623  1.00  3.95           H  
ATOM     17  H17                -2.629   9.049  -3.336  1.00  3.95           H  
ATOM     18  H18                -1.905   8.205  -2.185  1.00  3.95           H  
ATOM     19  O19                -2.416  10.887  -1.014  1.00  3.95           O  
ATOM     20  O20                -4.047   9.347  -1.125  1.00  3.95           O  
'''
prb = '''CRYST1   12.724   13.662   19.746  95.98 102.66 117.49 P -1          2
TITLE OLEX2 export (an aspirinate model derived from IVUYEE)
ATOM      1   O1                 3.330   0.446  13.419  1.00  3.95           O  
ATOM      2   O2                 3.274  -0.175  11.319  1.00  3.95           O  
ATOM      3   O3                 6.060   2.103  15.115  1.00  3.95           O  
ATOM      4   O4                 6.270   0.860  13.305  1.00  3.95           O  
ATOM      5   C5                 4.602   1.690  11.922  1.00  3.95           C  
ATOM      6   C6                 3.631   0.550  12.321  1.00  3.95           C  
ATOM      7   C7                 6.663   2.981  12.235  1.00  3.95           C  
ATOM      8   H8                 7.472   3.106  12.674  1.00  3.95           H  
ATOM      9   C9                 5.766   1.919  12.562  1.00  3.95           C  
ATOM     10  C10                 5.112   3.557  10.563  1.00  3.95           C  
ATOM     11  H11                 4.887   4.130   9.865  1.00  3.95           H  
ATOM     12  C12                 4.266   2.496  10.872  1.00  3.95           C  
ATOM     13  H13                 3.494   2.340  10.378  1.00  3.95           H  
ATOM     14  C14                 6.257   3.773  11.259  1.00  3.95           C  
ATOM     15  H15                 6.778   4.511  11.040  1.00  3.95           H  
ATOM     16  C16                 6.318   1.108  14.583  1.00  3.95           C  
ATOM     17  C17                 6.866  -0.202  15.317  1.00  3.95           C  
ATOM     18  H18                 7.014  -0.894  14.668  1.00  3.95           H  
ATOM     19  H19                 6.221  -0.501  15.963  1.00  3.95           H  
ATOM     20  H20                 7.691   0.002  15.761  1.00  3.95           H  
'''

# Test options reorder and keep_stereo

In [3]:
min_rmsd, prb_xyz, i_j_product = align(ref, prb, noHydrogens=True, reorder=False, keep_stereo=False)
print('mininum rmsd:',min_rmsd,'\n') 

mininum rmsd: 2.832728101125044 



In [4]:
min_rmsd, prb_xyz, i_j_product = align(ref, prb, noHydrogens=True, reorder=False, keep_stereo=True)
print('mininum rmsd:',min_rmsd,'\n') 

mininum rmsd: 2.832728101125044 



In [5]:
min_rmsd, prb_xyz, i_j_product = align(ref, prb, noHydrogens=True, reorder=True, keep_stereo=False)
print('rmsd', min_rmsd, '\ni_j_product: ',i_j_product, '\n\n', prb_xyz)

rmsd 0.1935363789423386 
i_j_product:  -1 

 13
aligned
O           -1.992           9.246           1.575
O           -0.790          10.837           2.483
C           -1.781          10.341           1.827
C           -2.776          11.422           1.334
C           -3.203          11.498           0.058
C           -4.143          12.464          -0.414
C           -4.610          13.285           0.509
C           -4.180          13.269           1.796
C           -3.237          12.343           2.232
C           -3.022           9.863          -1.471
C           -1.991           9.188          -2.489
O           -2.414          10.856          -0.886
O           -4.123           9.534          -1.332


In [6]:
min_rmsd, prb_xyz, i_j_product = align(ref, prb, noHydrogens=True, reorder=True, keep_stereo=True)
print('rmsd', min_rmsd, '\ni_j_product: ',i_j_product, '\n\n', prb_xyz)

rmsd 0.8691183513259775 
i_j_product:  1 

 13
aligned
O           -2.225           9.678           2.956
O           -0.949          10.075           1.220
C           -1.882          10.252           1.857
C           -2.915          11.306           1.382
C           -3.412          11.333           0.130
C           -4.340          12.311          -0.342
C           -4.668          13.239           0.539
C           -4.221          13.228           1.821
C           -3.344          12.247           2.274
C           -2.531          10.265          -1.640
C           -2.453           8.833          -2.347
O           -2.020          11.225          -2.036
O           -3.300          10.154          -0.594


# We see the option keep_stereo is responsible for the rmsd difference. 

My explanation: I mentioned the keep_stereo=True will dictate the algorithm to skip 24 enantiomers. Internally, there is one enatiomer which has 0.1935363789423386 RMSD against reference molecule and it correspond to the negative i_j_product. (see the output of 5th cell) 

My version of RMSD can also provide you an aligned probe molecule. You can view the xyz structure in the output of 6th cell (rmsd 0.869). It can be aligned to M2.pdb perfectly (which means it's the aligned probe molecule) but is the enatiomer of M1.pdb (which can not be superimposed with M1.pdb anyhow). This indicates keep_stereo=True would cause a problem if you care about the stereochemistry of moleclue and this is why I devise this option. It also saves the environment. 

BTW, It's pity I found Temelso2017.pdf before this repository. The code in Temelso's SI is really ugly... It takes me huge efforts to digest it. Sigh... 